In [1]:
import numpy as np 
import pandas as pd 
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from xgboost import XGBClassifier


warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

# Reading  file

In [2]:
df = pd.read_csv('/kaggle/input/loan-credit-risk-and-population-stability/loan_2019_20.csv')

In [3]:
df.shape

(664824, 142)

In [4]:
df.head(5)

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,0,149203043,24000.0,24000.0,24000.0,60 months,13.90%,557.20,C,C1,Budget analyst,10+ years,MORTGAGE,86000.0,Verified,2019-03-01,Current,n,https://lendingclub.com/browse/loanDetail.acti...,major_purchase,Major purchase,360xx,AL,23.54,1.0,Jan-2000,665.0,669.0,1.0,18.0,NaN,15.0,0.0,10899.0,73.1%,28.0,w,20427.69,20427.69,6649.33,6649.33,3572.31,3077.02,0.0,0.0,0.0,May-2020,0.00,Jul-2020,May-2020,659.0,655.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,234833.0,0.0,4.0,2.0,4.0,11.0,62839.0,73.0,2.0,3.0,2784.0,73.0,14900.0,6.0,6.0,7.0,8.0,15656.0,3112.0,70.1,0.0,0.0,134.0,230.0,11.0,10.0,3.0,11.0,NaN,5.0,NaN,0.0,5.0,9.0,6.0,7.0,14.0,10.0,11.0,9.0,15.0,0.0,0.0,0.0,5.0,92.9,66.7,0.0,0.0,255961.0,73738.0,10400.0,77261.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,CVD19SKIP,INCOMECURT,ACTIVE,2.0,0.0,Apr-2020,Jun-2020,Apr-2020,2.0,0.0,ACTIVE,473.24,20656.42,557.2,N
1,1,149354242,18500.0,18500.0,18500.0,60 months,14.74%,437.60,C,C2,NaN,NaN,MORTGAGE,43000.0,Verified,2019-03-01,Current,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,305xx,GA,19.73,0.0,Apr-1996,730.0,734.0,1.0,NaN,NaN,7.0,0.0,5417.0,24%,27.0,w,15615.82,15615.82,5658.50,5658.50,2884.18,2774.32,0.0,0.0,0.0,May-2020,437.60,Jun-2020,May-2020,789.0,785.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,22602.0,1.0,3.0,1.0,2.0,6.0,17185.0,77.0,1.0,2.0,499.0,51.0,22600.0,0.0,11.0,1.0,4.0,3229.0,11501.0,4.2,0.0,0.0,275.0,75.0,7.0,6.0,0.0,7.0,NaN,4.0,NaN,0.0,1.0,2.0,2.0,3.0,20.0,4.0,7.0,2.0,7.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,0.0,44790.0,22602.0,12000.0,22190.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,2,149355875,24000.0,24000.0,24000.0,36 months,8.19%,754.18,A,A4,Pilot,10+ 

# Creating Target for Classification

In [5]:
df['loan_status'].unique()

array(['Current', 'Fully Paid', 'Charged Off', 'Late (31-120 days)',
       'In Grace Period', 'Late (16-30 days)', 'Default', 'Issued'],
      dtype=object)

In [6]:
# Define good vs bad loan statuses
good_statuses = ['Fully Paid', 'Current', 'Issued', 'In Grace Period']
bad_statuses = ['Charged Off', 'Default', 'Late (31-120 days)', 'Late (16-30 days)']

# Create binary target column
def map_status(status):
    if status in good_statuses:
        return 0
    elif status in bad_statuses:
        return 1

df['target'] = df['loan_status'].apply(map_status)


df.drop(['loan_status'], axis=1, inplace=True)

# Missing Values 

Delete all columns with more than 40% missing values

In [7]:
missing = (df.isna().sum() / len(df)) * 100
cols_over_40 = missing[missing > 40].index.tolist()
df = df.drop(columns=cols_over_40)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664824 entries, 0 to 664823
Columns: 108 entries, Unnamed: 0 to target
dtypes: float64(81), int64(3), object(24)
memory usage: 547.8+ MB


# Categorical Columns

In [9]:
df.select_dtypes(include=['object']).head()

,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,pymnt_plan,url,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type,hardship_flag,debt_settlement_flag
0,60 months,13.90%,C,C1,Budget analyst,10+ years,MORTGAGE,Verified,2019-03-01,n,https://lendingclub.com/browse/loanDetail.acti...,major_purchase,Major purchase,360xx,AL,Jan-2000,73.1%,w,May-2020,Jul-2020,May-2020,Individual,Y,N
1,60 months,14.74%,C,C2,NaN,NaN,MORTGAGE,Verified,2019-03-01,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,305xx,GA,Apr-1996,24%,w,May-2020,Jun-2020,May-2020,Individual,N,N
2,36 months,8.19%,A,A4,Pilot,10+ years,MORTGAGE,Not Verified,2019-03-01,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,309xx,GA,Jul-1995,80%,w,May-2020,Jun-2020,May-2020,Individual,N,N
3,36 months,8.19%,A,A4,Brand and Content Strategist,2 years,RENT,Not Verified,2019-03-01,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Credit card refinancing,701xx,LA,Sep-2006,18.3%,f,May-2020,Jun-2020,May-2020,Individual,N,N
4,36 months,20.00%,D,D2,CARE GIVER,3 years,RENT,Not Verified,2019-03-01,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,904xx,CA,Oct-2014,47.4%,w,May-2020,Jun-2020,May-2020,Individual,N,N


In [10]:
df.select_dtypes(include='object').nunique().sort_values(ascending=False)

url                     664824
emp_title               160283
revol_util                1142
zip_code                   903
earliest_cr_line           717
int_rate                    81
addr_state                  50
sub_grade                   33
last_pymnt_d                24
last_credit_pull_d          24
issue_d                     21
next_pymnt_d                13
purpose                     12
title                       12
emp_length                  11
grade                        7
home_ownership               5
verification_status          3
hardship_flag                2
application_type             2
term                         2
initial_list_status          2
debt_settlement_flag         2
pymnt_plan                   1
dtype: int64

In [11]:
df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

df['int_rate'] = df['int_rate'].str.replace('%', '').str.strip().astype(float)/100

df['revol_util'] = df['revol_util'].str.replace('%', '').str.strip().astype(float)/100

def clean_years_exp(x):
    if pd.isna(x):
        return x  # keep NaN as is
    x = x.strip()  # remove extra spaces
    if x == '< 1 year' or x == '< 1 years':
        return 0
    elif '10+' in x:
        return 10
    else:
        # extract the number at the start
        num = ''.join([ch for ch in x if ch.isdigit()])
        return int(num)

df['emp_length'] = df['emp_length'].apply(clean_years_exp)
df['emp_length'].unique()

array([10., nan,  2.,  3.,  9.,  5.,  8.,  0.,  6.,  4.,  7.,  1.])

In [12]:
df['hardship_flag'] = (df['hardship_flag'] == 'Y').astype(int)
df['application_type'] = (df['application_type'] == 'Joint App').astype(int)
df['term'] = (df['term'] == '60 months').astype(int)
df['initial_list_status'] = (df['initial_list_status'] == 'f').astype(int)
df['debt_settlement_flag'] = df['debt_settlement_flag'].map({'N': 0, 'Y': 1})
df['home_ownership'] = df['home_ownership'].replace(['ANY', 'NONE'], 'OTHER')

In [13]:
# categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# for col in categorical_cols:
#     print("\n==============================")
#     print(f"Column: {col}")
#     print("==============================")

#     # Show value counts
#     print("\nValue Counts:")
#     print(df[col].value_counts())

#     # Show bad-rate relative to target
#     print("\nBad Rate by Category:")
#     print(df.groupby(col)['target'].mean())

In [14]:
# Convert 'issue_d' to datetime
df['issue_d'] = pd.to_datetime(df['issue_d'], errors='coerce')

# Convert 'earliest_cr_line' to datetime
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%b-%Y', errors='coerce')

df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'], format='%b-%Y', errors='coerce')
df['next_pymnt_d'] = pd.to_datetime(df['next_pymnt_d'], format='%b-%Y', errors='coerce')
df['last_credit_pull_d'] = pd.to_datetime(df['last_credit_pull_d'], format='%b-%Y', errors='coerce')

**High Cardinality and Redundant columns**

In [15]:
cols_to_drop = {'title','zip_code','pymnt_plan', 'emp_title', 'url', 'sub_grade'}

df.drop(cols_to_drop, axis = 1, inplace = True)

# DateTime Columns

In [16]:
date_cols = df.select_dtypes(include=['datetime64']).columns
df[date_cols].isna().sum()

issue_d                   0
earliest_cr_line          0
last_pymnt_d           2498
next_pymnt_d          79611
last_credit_pull_d        3
dtype: int64

In [17]:
# 1. last_pymnt_d → replace missing with issue date
df['last_pymnt_d'] = df['last_pymnt_d'].fillna(df['issue_d'])

# 2. next_pymnt_d → impute differently for good/bad loans
df.loc[df['next_pymnt_d'].isna(), 'next_pymnt_d'] = df['last_pymnt_d']

# 3. last_credit_pull_d → fill with max date
max_pull = df['last_credit_pull_d'].max()
df['last_credit_pull_d'] = df['last_credit_pull_d'].fillna(max_pull)

In [18]:
snapshot_date = pd.to_datetime(
    df[['issue_d','earliest_cr_line','last_pymnt_d','next_pymnt_d','last_credit_pull_d']].max().max()
)

In [19]:
df['loan_age_days'] = (snapshot_date - df['issue_d']).dt.days
df['credit_history_length_days'] = (df['issue_d'] - df['earliest_cr_line']).dt.days
df['days_since_last_payment'] = (snapshot_date - df['last_pymnt_d']).dt.days
df['days_until_next_payment'] = (df['next_pymnt_d'] - snapshot_date).dt.days
df['payment_overdue'] = (df['days_until_next_payment'] < 0).astype(int)
df['days_since_credit_pull'] = (snapshot_date - df['last_credit_pull_d']).dt.days
df['issue_year'] = df['issue_d'].dt.year
df['issue_month'] = df['issue_d'].dt.month
df['recent_payment_flag'] = (df['days_since_last_payment'] <= 30).astype(int)

In [20]:
drop_date = {'issue_d','earliest_cr_line','last_pymnt_d', 'next_pymnt_d','last_credit_pull_d'}
df.drop(drop_date, axis=1, inplace=True)

# Numerical Columns

In [21]:
df.describe()

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target,loan_age_days,credit_history_length_days,days_since_last_payment,days_until_next_payment,payment_overdue,days_since_credit_pull,issue_year,issue_month,recent_payment_flag
count,664824.000000,6.648240e+05,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,606511.000000,6.648240e+05,663427.000000,664824.00000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,6.648240e+05,663966.000000,664824.000000,664824.000000,664824.00000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.0,664824.000000,664824.000000,664824.000000,6.648240e+05,664824.000000,664824.000000,664824.00000,664824.000000,648219.000000,6.648240e+05,576526.000000,664824.000000,664824.000000,664824.000000,664687.000000,6.648240e+05,664824.000000,664824.000000,664824.000000,664824.000000,664782.000000,657091.000000,656754.000000,664824.000000,664824.000000,648219.000000,664824.000000,664824.000000,664824.000000,664824.000000,657535.000000,588354.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.00000,664824.000000,664824.000000,664824.000000,656795.000000,664824.000000,664824.000000,664824.000000,664824.000000,657054.000000,664824.000000,664824.0,6.648240e+05,6.648240e+05,6.648240e+05,6.648240e+05,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.000000,664824.0
mean,59788.513119,1.573627e+08,16419.169246,16419.169246,16416.641490,0.322643,0.128872,474.811153,5.463936,8.655052e+04,20.904595,0.23129,707.049407,711.049646,0.477289,11.892972,0.104050,1.800667e+04,0.443894,23.528009,0.091539,12215.02297,12213.304990,5140.746684,5139.819802,3889.889618,1238.272752,0.441525,12.142757,2.034325,1561.608489,713.097719,707.637555,0.016104,1.0,0.135078,0.000110,164.665053,1.548830e+05,0.924570,2.926171,0.74597,1.690327,18.695891,3.962250e+04,68.597255,1.177624,2.427205,6462.499693,54.697765,4.215279e+04,1.193379,1.602711,1.928067,4.388981,14253.914781,16965.692729,49.344697,0.006451,1.534714,126.442259,173.786210,15.897256,8.501627,1.369200,25.818019,7.387943,0.448028,3.808820,5.533780,5.125475,7.274610,8.922073,8.34193,13.039541,5.515952,11.877816,0.000018,0.000062,0.056920,2.053807,94.903059,32.216303,0.103939,0.0,1.997200e+05,5.809352e+04,2.998834e+04,5.103126e+04,0.099186,0.001089,0.032252,533.441788,5924.182671,287.014187,-255.978266,0.999994,278.439452,2019.220685,5.886895,0.0
std,38084.736456,7.141676e+06,10328.408685,10328.408685,10329.018233,0.467488,0.049528,290.622

In [22]:
df.nunique().sort_values(ascending=False)

id                     664824
tot_hi_cred_lim        328840
tot_cur_bal            311313
out_prncp_inv          304267
out_prncp              300985
                        ...  
hardship_flag               2
term                        2
policy_code                 1
tax_liens                   1
recent_payment_flag         1
Length: 106, dtype: int64

In [23]:
drop_num = {'policy_code','tax_liens','recent_payment_flag', 'Unnamed: 0','id'}
df.drop(drop_num, axis=1, inplace=True)

In [24]:
df["fico_orig"] = (df["fico_range_low"] + df["fico_range_high"]) / 2
df["fico_recent"] = (df["last_fico_range_low"] + df["last_fico_range_high"]) / 2

df["fico_change"] = df["fico_recent"] - df["fico_orig"]

df["fico_ratio"] = df["fico_recent"] / df["fico_orig"]
df["fico_drop_flag"] = (df["fico_change"] < -20).astype(int)
df["fico_improve_flag"] = (df["fico_change"] > 20).astype(int)

fico = {'last_fico_range_high','last_fico_range_low','fico_range_low', 'fico_range_high'}
df.drop(fico , axis=1, inplace=True)

Find missing variable and fill in

In [25]:
num_cols = df.select_dtypes(exclude=['object']).columns

df[num_cols].isna().sum()[df[num_cols].isna().sum() > 0]

emp_length               58313
dti                       1397
revol_util                 858
mths_since_rcnt_il       16605
il_util                  88298
all_util                   137
avg_cur_bal                 42
bc_open_to_buy            7733
bc_util                   8070
mo_sin_old_il_acct       16605
mths_since_recent_bc      7289
mths_since_recent_inq    76470
num_tl_120dpd_2m          8029
percent_bc_gt_75          7770
dtype: int64

In [26]:
# 1. Fill with -1
fill_minus1 = [
    'mths_since_rcnt_il',
    'mo_sin_old_il_acct',
    'mths_since_recent_bc',
    'mths_since_recent_inq'
]
df[fill_minus1] = df[fill_minus1].fillna(-1)

# 2. Ratio columns → median
fill_median_ratio = [
    'revol_util','il_util','all_util',
    'bc_util','percent_bc_gt_75'
]
for col in fill_median_ratio:
    df[col] = df[col].fillna(df[col].median())

# 3. Balances / dti → median
fill_median_balance = ['dti','avg_cur_bal','bc_open_to_buy']
for col in fill_median_balance:
    df[col] = df[col].fillna(df[col].median())

# 4. Delinquency counts → zero
df['num_tl_120dpd_2m'] = df['num_tl_120dpd_2m'].fillna(0)

# 5. Emp length → mode
df['emp_length'] = df['emp_length'].fillna(df['emp_length'].mode()[0])

# Feature Engineering Using Correlation

In [27]:
corr_matrix = df[num_cols].corr().abs()

# Keep only upper triangle (no self-corr or duplicate pairs)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Filter by threshold
threshold = 0.80
high_corr = upper.stack().sort_values(ascending=False)
high_corr = high_corr[high_corr > threshold]

high_corr

loan_amnt                funded_amnt                   1.000000
total_pymnt              total_pymnt_inv               0.999998
out_prncp                out_prncp_inv                 0.999996
funded_amnt              funded_amnt_inv               0.999994
loan_amnt                funded_amnt_inv               0.999994
open_acc                 num_sats                      0.999691
pub_rec                  pub_rec_bankruptcies          0.999267
fico_change              fico_ratio                    0.998904
num_actv_rev_tl          num_rev_tl_bal_gt_0           0.994191
days_since_last_payment  days_until_next_payment       0.986800
tot_cur_bal              tot_hi_cred_lim               0.979133
total_pymnt              total_rec_prncp               0.978623
total_pymnt_inv          total_rec_prncp               0.978604
total_bal_il             total_il_high_credit_limit    0.945746
loan_amnt                installment                   0.938199
funded_amnt              installment    

In [28]:
to_drop = set()
target_corr = df[num_cols].corr()['target'].abs()

for (feat1, feat2), corr_val in high_corr.items():
    
    # If either feature is already dropped, skip
    if feat1 in to_drop or feat2 in to_drop:
        continue
    
    # Compare correlation with target
    corr1 = target_corr.get(feat1, 0)
    corr2 = target_corr.get(feat2, 0)
    
    # Drop the one with LOWER correlation to target
    if corr1 < corr2:
        to_drop.add(feat1)
    else:
        to_drop.add(feat2)

to_drop

{'bc_open_to_buy',
 'bc_util',
 'credit_history_length_days',
 'days_until_next_payment',
 'fico_change',
 'funded_amnt',
 'funded_amnt_inv',
 'last_pymnt_amnt',
 'loan_amnt',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'open_acc',
 'open_rv_12m',
 'open_rv_24m',
 'out_prncp_inv',
 'pub_rec_bankruptcies',
 'recoveries',
 'revol_util',
 'tot_cur_bal',
 'total_bal_ex_mort',
 'total_bal_il',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rev_hi_lim'}

In [29]:
#important derived columns
df['loan_amnt_div_instlmnt']=df['loan_amnt']/df['installment']
df['pct_rev_active'] = df['num_rev_tl_bal_gt_0'] / df['num_rev_accts']

In [30]:
df.drop(to_drop, axis=1, inplace=True)

In [31]:
df['target'].value_counts()

target
0    643382
1     21442
Name: count, dtype: int64

In [32]:
y = df['target']

# Identify columns
cat_cols = df.select_dtypes(include=['object']).columns
num_cols = df.select_dtypes(exclude=['object']).drop('target', axis=1).columns

# OneHotEncode categorical columns
ohe = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')
encoded = ohe.fit_transform(df[cat_cols])

# Convert encoded array to dataframe
encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(cat_cols), index=df.index)

# Combine numeric + encoded categorical
X = pd.concat([df[num_cols], encoded_df], axis=1)


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Selection with RandomForestClassifer

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# 1. Fit the random forest
rf = RandomForestClassifier(
    n_estimators=150,
    random_state=42,
)
rf.fit(X_train, y_train)

# 2. Use SelectFromModel with mean importance
selector = SelectFromModel(
    rf,
    threshold="mean",  # mean importance threshold
    prefit=True        # RF is already fitted
)

# 3. Get mask of selected features
selected_mask = selector.get_support()

# 4. Get names of selected features
selected_features = X_train.columns[selected_mask].tolist()

selected_features


['installment',
 'out_prncp',
 'total_rec_prncp',
 'total_rec_int',
 'collection_recovery_fee',
 'hardship_flag',
 'loan_age_days',
 'days_since_last_payment',
 'days_since_credit_pull',
 'fico_recent',
 'fico_ratio',
 'fico_drop_flag',
 'loan_amnt_div_instlmnt']

In [36]:
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

array([[5.99480000e+02, 0.00000000e+00, 2.80000000e+04, ...,
        9.50495050e-01, 1.00000000e+00, 4.67071462e+01],
       [2.61630000e+02, 0.00000000e+00, 7.40000000e+03, ...,
        1.00712251e+00, 0.00000000e+00, 2.82842182e+01],
       [3.14250000e+02, 6.97729000e+03, 3.02271000e+03, ...,
        9.85754986e-01, 0.00000000e+00, 3.18217979e+01],
       ...,
       [4.59470000e+02, 1.11197300e+04, 3.88027000e+03, ...,
        1.04213483e+00, 0.00000000e+00, 3.26463099e+01],
       [6.73420000e+02, 2.59152700e+04, 4.08473000e+03, ...,
        1.00749625e+00, 0.00000000e+00, 4.45487215e+01],
       [1.68670000e+02, 3.81193000e+03, 1.18807000e+03, ...,
        1.00755287e+00, 0.00000000e+00, 2.96436829e+01]])

# Model Training

In [37]:
model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=30,
    eval_metric="aucpr",
    tree_method="hist",
    random_state=42
)

model.fit(X_train_selected, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='aucpr', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

# Model Evaluation

In [40]:
from sklearn.metrics import average_precision_score

# ==== 1. Predictions ====
y_pred = model.predict(X_test_selected)
y_prob = model.predict_proba(X_test_selected)[:, 1]

# ==== 2. Accuracy ====
print("Accuracy:", accuracy_score(y_test, y_pred))

# ==== 3. Confusion Matrix ====
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cm)

# ==== 4. Classification Report ====
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# ==== 5. ROC-AUC (Recommended metric #1) ====
auc = roc_auc_score(y_test, y_prob)
print("ROC AUC:", auc)

# ==== 6. PR-AUC (Recommended metric #2 for imbalance) ====
pr_auc = average_precision_score(y_test, y_prob)
print("Precision-Recall AUC:", pr_auc)

Accuracy: 0.9905915090437333

Confusion Matrix:
 [[127480   1052]
 [   199   4234]]

Classification Report:

              precision    recall  f1-score   support

           0       1.00      0.99      1.00    128532
           1       0.80      0.96      0.87      4433

    accuracy                           0.99    132965
   macro avg       0.90      0.97      0.93    132965
weighted avg       0.99      0.99      0.99    132965

ROC AUC: 0.9915141168393778
Precision-Recall AUC: 0.9650417831336116


# Save Model

In [42]:
import joblib

joblib.dump(model, "/kaggle/working/model.pkl")

['/kaggle/working/model.pkl']